# 🌿 PhytoChem-Tracker v2.0

**Version:** 2.0 (Stable)  
**Author:** rpwdoc2025  
**Engine:** Google Gemini 3.0 + Agentic Workflow

## Overview
PhytoChem-Tracker is an AI-driven platform for biochemists. It simulates an agentic workflow to retrieve, rank, and analyze phytochemical data from sources like PubChem, GNPS, and Phytozome, synthesized by Gemini 3.0.

### 🚀 How to Launch
1. **Setup:** Click the 'Play' button on the first cell to install tools.
2. **Authenticate:** Enter your Google Gemini API Key.
3. **Run:** Execute the Dashboard cell to start the interface.

In [ ]:
# @title 1. System Setup
# Installs: Google GenAI SDK, PDF Generator, Data Tools, UI Widgets
!pip install -q google-generativeai ipywidgets fpdf pandas

In [ ]:
# @title 2. Load Agents & Data
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import google.generativeai as genai
import pandas as pd
from fpdf import FPDF
import time
from datetime import datetime

# --- DATASETS (MOCK DATABASE) ---
MOCK_TEA_DATA = [
    {"Rank": 1, "Name": "Epigallocatechin gallate (EGCG)", "Formula": "C22H18O11", "MW": 458.37, "Papers": 14502, "SMILES": "C1[C@@H]([C@H](OC2=CC(=CC(=C21)O)O)C3=CC(=C(C(=C3)O)O)O)OC(=O)C4=CC(=C(C(=C4)O)O)O"},
    {"Rank": 2, "Name": "Caffeine", "Formula": "C8H10N4O2", "MW": 194.19, "Papers": 32100, "SMILES": "CN1C=NC2=C1C(=O)N(C(=O)N2C)C"},
    {"Rank": 3, "Name": "L-Theanine", "Formula": "C7H14N2O3", "MW": 174.20, "Papers": 1250, "SMILES": "CCN=C(O)CC[C@H](N)C(=O)O"},
    {"Rank": 4, "Name": "Kaempferol", "Formula": "C15H10O6", "MW": 286.23, "Papers": 8900, "SMILES": "C1=CC(=CC=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O"},
    {"Rank": 5, "Name": "Quercetin", "Formula": "C15H10O7", "MW": 302.23, "Papers": 22100, "SMILES": "C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O"}
]

MOCK_GENERIC_DATA = [
    {"Rank": 1, "Name": "Simulated Compound A", "Formula": "C10H12O4", "MW": 196.2, "Papers": 50, "SMILES": "CC(C)C1=CC=C(C=C1)O"},
    {"Rank": 2, "Name": "Simulated Compound B", "Formula": "C12H22O11", "MW": 342.3, "Papers": 12, "SMILES": "OCC1OC(CO)C(O)C1O"}
]

# --- PDF REPORT ENGINE ---
class ReportPDF(FPDF):
    def header(self):
        self.set_fill_color(16, 185, 129) # Emerald Green
        self.rect(0, 0, 210, 20, 'F')
        self.set_font('Arial', 'B', 15)
        self.set_text_color(255, 255, 255)
        self.cell(0, 5, 'PhytoChem-Tracker v2.0 Report', 0, 1, 'L')
        self.ln(15)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

def generate_pdf(species, summary, data, filename="PhytoChem_Report.pdf"):
    pdf = ReportPDF()
    pdf.add_page()
    
    # Title Info
    pdf.set_text_color(0)
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, f"Target: {species}", 0, 1)
    pdf.set_font("Arial", "", 10)
    pdf.set_text_color(100)
    pdf.cell(0, 10, f"Analysis Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", 0, 1)
    
    # Gemini Summary Section
    pdf.ln(5)
    pdf.set_text_color(0)
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "AI Executive Summary (Gemini 3.0)", 0, 1)
    pdf.set_font("Arial", "", 10)
    pdf.multi_cell(0, 6, summary)
    
    # Data Table
    pdf.ln(10)
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, "Identified Phytochemicals", 0, 1)
    
    # Table Header
    pdf.set_fill_color(240, 240, 240)
    pdf.set_font("Arial", "B", 9)
    pdf.cell(15, 8, "Rank", 1, 0, 'C', 1)
    pdf.cell(65, 8, "Name", 1, 0, 'L', 1)
    pdf.cell(40, 8, "Formula", 1, 0, 'L', 1)
    pdf.cell(25, 8, "MW", 1, 0, 'L', 1)
    pdf.cell(30, 8, "Citations", 1, 1, 'C', 1)
    
    # Table Rows
    pdf.set_font("Arial", "", 8)
    for item in data:
        pdf.cell(15, 8, str(item['Rank']), 1, 0, 'C')
        pdf.cell(65, 8, item['Name'][:35], 1, 0, 'L')
        pdf.cell(40, 8, item['Formula'], 1, 0, 'L')
        pdf.cell(25, 8, str(item['MW']), 1, 0, 'L')
        pdf.cell(30, 8, str(item['Papers']), 1, 1, 'C')
        
    pdf.output(filename)
    return filename

# --- GEMINI INTELLIGENCE ---
def run_gemini_analysis(api_key, query, count):
    if not api_key:
        return "[System] API Key missing. Please provide a key for real-time analysis."
    try:
        genai.configure(api_key=api_key)
        # Fallback to 'gemini-pro' if specific version names change
        model = genai.GenerativeModel('gemini-1.5-flash') 
        
        prompt = f"""
        Act as the PhytoChem-Tracker engine. The user is analyzing: '{query}'.
        We have found {count} major compounds.
        Write a 3-sentence scientific summary of the therapeutic value and industrial applications of these phytochemicals.
        """
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Gemini Connection Error: {str(e)}"

print("✅ System Loaded: Agents Ready.")

In [ ]:
# @title 3. PhytoChem-Tracker v2.0 Dashboard

# --- UI LAYOUT & WIDGETS ---
style = {'description_width': 'initial'}
layout_full = widgets.Layout(width='98%')

header = widgets.HTML(
    "<div style='background:#0f172a; padding:15px; border-radius:10px; color:white;'>"
    "<h2 style='margin:0; color:#10b981'>🌿 PhytoChem-Tracker v2.0</h2>"
    "<p style='margin:5px 0 0 0; font-size:12px; opacity:0.8'>Agentic AI for Biochemical Intelligence</p>"
    "</div>"
)

key_input = widgets.Password(
    placeholder='Enter Google Gemini API Key',
    description='<b>🔑 API Key:</b>',
    style=style, layout=layout_full
)

query_input = widgets.Text(
    value='Camellia sinensis',
    placeholder='Enter species name (e.g., Curcuma longa)',
    description='<b>🔍 Target:</b>',
    style=style, layout=layout_full
)

search_depth = widgets.ToggleButtons(
    options=['Rapid Scan', 'Deep Literature', 'Genomic Map'],
    description='<b>Depth:</b>',
    style=style,
    button_style='info'
)

run_btn = widgets.Button(
    description=' START ANALYSIS',
    button_style='success',
    layout=widgets.Layout(width='100%', height='50px'),
    icon='rocket'
)

log_console = widgets.Output(layout={'border': '1px solid #333', 'height': '160px', 'overflow_y': 'scroll', 'background_color': '#1e293b', 'color': '#00ff9d', 'padding': '10px'})
results_area = widgets.Output()

# --- LOGIC ---
def log(msg):
    with log_console:
        print(f"> {datetime.now().strftime('%H:%M:%S')} | {msg}")

def execute_analysis(b):
    log_console.clear_output()
    results_area.clear_output()
    
    target = query_input.value
    api_key = key_input.value
    
    run_btn.disabled = True
    run_btn.description = "AGENTS WORKING..."
    
    try:
        log("Orchestrator: Initializing sequence...")
        time.sleep(0.5)
        log(f"Target Agent: Locked on '{target}'")
        
        # 1. Data Retrieval Simulation
        data = MOCK_GENERIC_DATA
        if "tea" in target.lower() or "camellia" in target.lower():
            data = MOCK_TEA_DATA
            
        log(f"Database Agent: Retrieved {len(data)} compound records (PubChem/GNPS).")
        time.sleep(1)
        
        # 2. AI Analysis
        log("Gemini Agent: Generating therapeutic insights...")
        summary = run_gemini_analysis(api_key, target, len(data))
        log("Gemini Agent: Insight generation complete.")
        
        # 3. Report Generation
        log("Report Agent: Compiling PDF...")
        pdf_file = generate_pdf(target, summary, data)
        log(f"System: Analysis Finished. Report saved as {pdf_file}.")
        
        # 4. Display Results
        with results_area:
            # Summary Card
            display(HTML(f"<div style='background:#ecfdf5; padding:15px; border-left:4px solid #10b981; color:#064e3b; margin-bottom:10px'><b>Gemini Insight:</b><br>{summary}</div>"))
            
            # Table
            df = pd.DataFrame(data)
            display(df)
            
            # Download Button
            display(HTML(f"<br><a href='{pdf_file}' target='_blank' style='background:#2563eb; color:white; padding:10px 20px; text-decoration:none; border-radius:5px; font-weight:bold'>⬇️ Download PDF Report</a>"))
            
    except Exception as e:
        log(f"CRITICAL ERROR: {str(e)}")
    
    finally:
        run_btn.disabled = False
        run_btn.description = " START ANALYSIS"

run_btn.on_click(execute_analysis)

ui = widgets.VBox([header, key_input, query_input, search_depth, run_btn, widgets.HTML("<b>System Logs:</b>"), log_console, widgets.HTML("<hr>"), results_area])
display(ui)